# Notebook #3: Heat exchanger design optimization with linear and hyperplane tree surrogates

This notebook is part of the Supporting Information of the paper "Hyperplane Decision Trees as Piecewise Linear Surrogate Models for Chemical Process Design", by Sunshine et al.

Notebook #2 showed details into training surrogate models for PHT tables, using mainly the [hyperplanetree](https://github.com/LLNL/systems2atoms/tree/add-hyperplanetree/systems2atoms/hyperplanetree) Python package. In the present Notebook, we focus in showing the embeding of the surrogates in the optimization problem of a heat exchanger design. 

Contact the authors of the paper if you have any questions.

## Installing the package from github

In [1]:
# pip install git+https://github.com/cog-imperial/OMLT.git --upgrade


In [2]:
# !pip install git+https://github.com/LLNL/systems2atoms

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from systems2atoms.hyperplanetree import LinearTreeRegressor, HyperplaneTreeRegressor, plot_surrogate_2d
import torch 
import sklearn
import time
from tqdm.auto import tqdm
from IPython.display import display, HTML
import base64
import io
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import lineartree
from sklearn.linear_model import LinearRegression
import matplotlib.patches as mpatches
from sklearn.metrics import mean_absolute_error

from systems2atoms.hyperplanetree import HyperplaneTreeDefinition, HyperplaneTreeHybridBigMFormulation, HyperplaneTreeGDPFormulation
from omlt.linear_tree import LinearTreeGDPFormulation, LinearTreeDefinition, LinearTreeHybridBigMFormulation

plt.rcParams['figure.dpi'] = 200
torch_device = 'cpu' 

## Preparing the data for use in the regression

In [4]:
df = pd.read_csv('PHTSV_Table_HMAX_Adjusted.csv', usecols=[0, 1, 2])

df

,P(Pa),H(J/mol),T(K)
0,1.000000e+05,1275.88638,290.000000
1,3.515152e+05,1275.88638,289.942853
2,6.030303e+05,1275.88638,289.885698
3,8.545455e+05,1275.88638,289.828536
4,1.106061e+06,1275.88638,289.771366
...,...,...,...
9995,2.399394e+07,65523.00000,920.211096
9996,2.424545e+07,65523.00000,920.898379
9997,2.449697e+07,65523.00000,921.583218
9998,2.474848e+07,65523.00000,922.265623


In [5]:
# Features will be the first two columns, pressure and enthalpy data
features = torch.tensor(df[['P(Pa)', 'H(J/mol)']].values, dtype=torch.float64)

# Response will be the third and last column, of temperature
y = torch.tensor(df['T(K)'].values, dtype=torch.float64)

features[:, 0] = features[:, 0] / 1e5  # Convert pressure from Pa to bar for consistency with plot in Ammari (20223)
features[:, 1] = features[:, 1]  / 1000  # Convert enthalpy from J/mol to kJ/mol 

pressure = features[:, 0]
enthalpy = features[:, 1]
temperature = df['T(K)']

### Getting train and test splits

Here, we use the splitting feature in [scikit-learn](https://scikit-learn.org/stable/). Alternative splitting methodologies can be used as well.

In [6]:
indices = np.arange(len(features))

# 20% of the data will be used for test, and 42 is used as the seed for random state, for reproducibility
train_features, test_features, train_y, test_y, train_indices, test_indices= train_test_split(features, y, indices, test_size=0.2, random_state=42)

# Making sure we have shuffled indices and correct splits
print(f'Train indices: {train_indices} ({np.size(train_indices)} points)')
print(f'Test indices: {test_indices} ({np.size(test_indices)} points)')

Train indices: [9254 1561 1670 ... 5390  860 7270] (8000 points)
Test indices: [6252 4684 1731 ... 7853 1095 6929] (2000 points)


## Training the different tree surrogates

We now train three surrogates. The first (1) is a linear model decision tree. The second (2) is a hyperplane tree model with weight = 2, and the third (3) is a hyperplane tree model with weight = 2. We use these to show how much the hyperparameters influence final results especially regarding training and optimization run times. 

### LMDT as used by [Ammari et al](https://www.sciencedirect.com/science/article/pii/S009813542300217X), with the package by [Cerliani](https://github.com/cerlymarco/linear-tree).

In [7]:
model_ltr_ammari_cerliani = lineartree.LinearTreeRegressor(LinearRegression(), 
                                                           criterion="mae", 
                                                           min_samples_leaf=0.002, 
                                                           min_impurity_decrease = -np.inf, 
                                                           max_depth = 20, 
                                                           max_bins = 50)

t1 = time.time()
model_ltr_ammari_cerliani.fit(train_features, train_y)
t2 = time.time()

y_pred = model_ltr_ammari_cerliani.predict(test_features)
linear_error = mean_absolute_error(test_y, y_pred)

linear_time = t2 - t1

tree_summary = model_ltr_ammari_cerliani.summary()

linear_leaves = sum(1 for node_info in tree_summary.values() if 'children' not in node_info)


print(f'Model                  # Leaves      Error (MAE)    Training time (s)')
print(f'Hyperplane Tree:       {linear_leaves}            {linear_error:.3f}          {linear_time:.2f}')

Current number of nodes: 336 | Current number of leaves: 337
Model                  # Leaves      Error (MAE)    Training time (s)
Hyperplane Tree:       337            0.303          2.99


### Hyperplane tree, with maximum hyperplane weight = 2

In [8]:
model_ht_2 = HyperplaneTreeRegressor(
    min_samples_leaf = 0.0043,
        max_bins = 50,
        max_weight = 2,
        disable_tqdm = True,
        min_impurity_decrease = -np.inf,
)

t1 = time.time()
model_ht_2.fit(train_features, train_y)
t2 = time.time()

y_pred_ht = model_ht_2.predict(test_features.to(torch_device)).cpu()

hyperplane_error_2 = torch.mean(torch.abs(y_pred_ht - test_y))
hyperplane_leaves_2 = len(model_ht_2._leaves)
hyperplane_time_2 = t2 - t1

print(f'Model                  # Leaves      Error (MAE)    Training time (s)')
print(f'Hyperplane Tree:       {hyperplane_leaves_2}            {hyperplane_error_2:.3f}          {hyperplane_time_2:.2f}')

Model                  # Leaves      Error (MAE)    Training time (s)
Hyperplane Tree:       174            0.277          2.61


### Hyperplane tree, with maximum hyperplane weight = 3

In [9]:
df = pd.read_csv('PHTSV_Table_HMAX_Adjusted.csv', usecols=[0, 1, 2])


# Features will be the first two columns, pressure and enthalpy data
features = torch.tensor(df[['P(Pa)', 'H(J/mol)']].values, dtype=torch.float64)

# Response will be the third and last column, of temperature
y = torch.tensor(df['T(K)'].values, dtype=torch.float64)

features[:, 0] = features[:, 0] / 1e5  # Convert pressure from Pa to bar for consistency with plot in Ammari (20223)
features[:, 1] = features[:, 1]  / 1000  # Convert enthalpy from J/mol to kJ/mol 

pressure = features[:, 0]
enthalpy = features[:, 1]
temperature = df['T(K)']


indices = np.arange(len(features))

# 20% of the data will be used for test, and 42 is used as the seed for random state, for reproducibility
train_features, test_features, train_y, test_y, train_indices, test_indices= train_test_split(features, y, indices, test_size=0.2, random_state=42)

# Making sure we have shuffled indices and correct splits
print(f'Train indices: {train_indices} ({np.size(train_indices)} points)')
print(f'Test indices: {test_indices} ({np.size(test_indices)} points)')



model_ht_3 = HyperplaneTreeRegressor(
    min_samples_leaf = 0.0043,
        max_bins = 50,
        max_weight = 3,
        disable_tqdm = True,
        min_impurity_decrease = -np.inf,
)

t1 = time.time()
model_ht_3.fit(train_features, train_y)
t2 = time.time()

y_pred_ht = model_ht_3.predict(test_features.to(torch_device)).cpu()

hyperplane_error_3 = torch.mean(torch.abs(y_pred_ht - test_y))
hyperplane_leaves_3 = len(model_ht_3._leaves)
hyperplane_time_3 = t2 - t1

print(f'Model                  # Leaves      Error (MAE)    Training time (s)')
print(f'Hyperplane Tree:       {hyperplane_leaves_3}            {hyperplane_error_3:.3f}          {hyperplane_time_3:.2f}')

Train indices: [9254 1561 1670 ... 5390  860 7270] (8000 points)
Test indices: [6252 4684 1731 ... 7853 1095 6929] (2000 points)
Model                  # Leaves      Error (MAE)    Training time (s)
Hyperplane Tree:       168            0.302          6.94


### Optimization framework using the tree surrogates

In [10]:
import pyomo.environ as pyo
from omlt import OmltBlock

def create_model(surrogate_def, surrogate_formulation, model, **surrogate_params):
    'This function generalizes the creation of a Pyomo model that admits a surrogate model block'

    m = pyo.ConcreteModel()

    # Define parameters
    U = 6341.4
    Cp = 1.507
    Ks = 20
    Ka = 150

    # Define variables
    m.TsIn = pyo.Var()
    m.TsOut = pyo.Var()
    m.TpIn = pyo.Var()
    m.TpOut = pyo.Var()
    m.Fp = pyo.Var(within=pyo.NonNegativeReals)
    m.Fs = pyo.Var(within=pyo.NonNegativeReals)
    m.A = pyo.Var(within=pyo.NonNegativeReals)
    m.PsIn = pyo.Var(within=pyo.NonNegativeReals, bounds=(1, 250))
    m.PsOut = pyo.Var(within=pyo.NonNegativeReals, bounds=(1, 250))
    m.Q = pyo.Var(within=pyo.NonNegativeReals)
    m.HsIn = pyo.Var(within=pyo.NonNegativeReals, bounds=(1.275886, 65.523))
    m.HsOut = pyo.Var(within=pyo.NonNegativeReals, bounds=(1.275886, 65.523))
    m.LMTD = pyo.Var()
    m.dT1 = pyo.Var(bounds=(0.01, None))
    m.dT2 = pyo.Var(bounds=(0.01, None))

    # Constraints for the model - physical laws and design equations for the HEX
    m.heat_transfer = pyo.Constraint(expr=m.Q == U * m.A * m.LMTD)
    m.LMTD_chenApp = pyo.Constraint(expr=m.LMTD == (m.dT1 * m.dT2 * (m.dT1 + m.dT2) / 2) ** (1 / 3))
    m.dT1_eq = pyo.Constraint(expr=m.dT1 == m.TsOut - m.TpIn)
    m.dT2_eq = pyo.Constraint(expr=m.dT2 == m.TsIn - m.TpOut)
    m.proc_fluid_thermo = pyo.Constraint(expr=m.Q == m.Fp * Cp * (m.TpOut - m.TpIn))
    m.steam_thermo = pyo.Constraint(expr=m.Q == -m.Fs * (m.HsOut - m.HsIn) * 1000 / 18)

    # Surrogate block - this is where we embed the surrogate
    m.tree_outlet = OmltBlock()
    surrogate_instance = surrogate_def(model, **surrogate_params)
    formulation_instance = surrogate_formulation(surrogate_instance, 'hull')
    m.tree_outlet.build_formulation(formulation_instance)

    # Connect surrogate inputs and outputs
    m.connect_input_outlet_P = pyo.Constraint(expr=m.PsOut == m.tree_outlet.inputs[0])
    m.connect_input_outlet_H = pyo.Constraint(expr=m.HsOut == m.tree_outlet.inputs[1])
    m.connect_output_outlet = pyo.Constraint(expr=m.TsOut == m.tree_outlet.outputs[0])

    # Objective function - related to minimizing cost od HEX design
    m.obj = pyo.Objective(expr=Ks * m.Fs + Ka * m.A)

    # Fix variables
    m.PsIn.fix(86.047)
    m.PsOut.fix(86.047)
    m.TsIn.fix(866)
    m.TpIn.fix(513.15)
    m.TpOut.fix(831)
    m.Fp.fix(310 * 3600)
    m.HsIn.fix(65.233)

    return m

In [11]:
# Define the models
m1 = create_model(
    surrogate_def=LinearTreeDefinition,
    surrogate_formulation=LinearTreeGDPFormulation,
    model=model_ltr_ammari_cerliani,
    unscaled_input_bounds={0: (1, 250), 1: (1.275886, 65.523)}
)

m2 = create_model(
    surrogate_def=HyperplaneTreeDefinition,
    surrogate_formulation=HyperplaneTreeGDPFormulation,
    model=model_ht_2,
    input_bounds_matrix=torch.tensor([[1, 250], [1.275886, 65.523]], dtype=torch.float64)
)

m3 = create_model(
    surrogate_def=HyperplaneTreeDefinition,
    surrogate_formulation=HyperplaneTreeGDPFormulation,
    model=model_ht_3,
    input_bounds_matrix=torch.tensor([[1, 250], [1.275886, 65.523]], dtype=torch.float64)
)

# Use the global solver BARON - here we used the GAMS interface 
solver = pyo.SolverFactory("gams:baron")


# Solve m1 and measure time
start_time_m1 = time.time()
results1 = solver.solve(m1, tee=True)
end_time_m1 = time.time()
time_m1 = end_time_m1 - start_time_m1

# Solve m2 and measure time
start_time_m2 = time.time()
results2 = solver.solve(m2, tee=True)
end_time_m2 = time.time()
time_m2 = end_time_m2 - start_time_m2


# Solve m3 and measure time
start_time_m3 = time.time()
results3 = solver.solve(m3, tee=True)
end_time_m3 = time.time()
time_m3 = end_time_m3 - start_time_m3

m3_obj = pyo.value(m3.obj)
m3_Fs_kg_s = m3.Fs.value / 3600
m3_area = m3.A.value

WARNING (W1002): Setting Var 'tree_outlet.scaled_inputs[0]' to a numeric value
`0` outside the bounds (1.0, 250.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'tree_outlet.scaled_inputs[0]' to a numeric value
`0` outside the bounds (1.0, 250.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'tree_outlet.scaled_inputs[1]' to a numeric value
`0` outside the bounds (1.275886, 65.523).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'tree_outlet.scaled_inputs[0]' to a numeric value
`0` outside the bounds (1.0, 250.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'tree_outlet.scaled_inputs[0]' to a numeric value
`0` outside the bounds (1.0, 250.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'tree_outlet.scaled_inputs[1]' to a

In [12]:
# Original formulation results as mentioned in Ammari (2023)
original_obj = 4.78e6  #Value for the objective
original_Fs = 60.31   # Value for steam flow (kg/s)
original_A = 2924     # Value for heat exchanger area (m2)

m1_obj = m1.obj.expr()
m1_Fs = m1.Fs.value / 3600  # Convert to kg/s
m1_A = m1.A.value

m2_obj = m2.obj.expr()
m2_Fs = m2.Fs.value / 3600  # Convert to kg/s
m2_A = m2.A.value

m3_obj = m3.obj.expr()
m3_Fs = m3.Fs.value / 3600  # Convert to kg/s
m3_A = m3.A.value

### Results comparison only considering tree surrogates

In [13]:
comparison_df = pd.DataFrame({
    "Metric": [
        "Objective Value ($)", 
        "Steam Flow (kg/s)", 
        "Heat Exchanger Area (m2)", 
        "Run time (s)", 
        "# of leaves", 
        "MAE (K)",
        "Training time (s)",
        "BARON Memory usage (MB)"
    ],
    "Original formulation": [
        f"{original_obj:.2e}", 
        f"{original_Fs:.2f}", 
        f"{original_A:.0f}", 
        "-", 
        "-", 
        "-", 
        "-",
        "-",
    ],
    "For linear tree surrogate": [
        f"{m1_obj:.2e}", 
        f"{m1_Fs:.2f}", 
        f"{m1_A:.0f}", 
        f"{time_m1:.2f}", 
        f"{linear_leaves}", 
        f"{linear_error:.2f}",
        f"{linear_time:.2f}",
        6 
    ],
    "For hyperplane tree surrogate (W = 3)": [
        f"{m3_obj:.2e}", 
        f"{m3_Fs:.2f}", 
        f"{m3_A:.0f}", 
        f"{time_m3:.2f}", 
        f"{hyperplane_leaves_3}", 
        f"{hyperplane_error_3:.2f}",
        f"{hyperplane_time_3:.2f}",
        8 
    ],
        "For hyperplane tree surrogate (W = 2)": [
        f"{m2_obj:.2e}", 
        f"{m2_Fs:.2f}", 
        f"{m2_A:.0f}", 
        f"{time_m2:.2f}", 
        f"{hyperplane_leaves_2}", 
        f"{hyperplane_error_2:.2f}",
        f"{hyperplane_time_2:.2f}",
        11 
    ]
})

comparison_df


,Metric,Original formulation,For linear tree surrogate,For hyperplane tree surrogate (W = 3),For hyperplane tree surrogate (W = 2)
0,Objective Value ($),4.78e+06,4.76e+06,4.76e+06,4.76e+06
1,Steam Flow (kg/s),60.31,60.08,60.09,60.09
2,Heat Exchanger Area (m2),2924,2915,2919,2921
3,Run time (s),-,1.33,6.12,2.33
4,# of leaves,-,337,168,174
5,MAE (K),-,0.30,0.30,0.28
6,Training time (s),-,2.99,6.94,2.61
7,BARON Memory usage (MB),-,6,8,11


## Artificial Neural Network (ANN) as surrogate

With the ReLU activation function. Therefore, the surrogate is also piecewise.

### Hyperparameter optimization

Using scikit-learn for grid search of ideal hyperparameters, to achieve a low MAE value.

In [14]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

# Prepare data 
X_train = train_features.numpy()
y_train = train_y.numpy().flatten()
X_test = test_features.numpy()
y_test = test_y.numpy().flatten()

# Scale features and target
scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_x.fit_transform(X_train)
X_test_scaled = scaler_x.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).ravel()

# Parameter grid
param_grid = {
    'hidden_layer_sizes': [
        (100,), (150,), (100, 50), (150, 75),
        (128, 64, 32), (200, 100, 50), (200, 100, 50, 25)
    ],
    'activation': ['relu'],
    'solver': ['adam'],
    'alpha': [1e-6, 1e-5, 1e-4, 1e-3],  
    'learning_rate_init': [0.001, 0.0005, 0.0003, 0.0001],
    'batch_size': [32, 64, 128],
    'max_iter': [500], 
    'early_stopping': [True],
}

# Grid search
mlp = MLPRegressor(random_state=42)
grid_search = GridSearchCV(
    mlp,
    param_grid,
    cv=3,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=2
)
grid_search.fit(X_train_scaled, y_train_scaled)

# Results
print("Best Parameters:")
print(grid_search.best_params_)

# Evaluate best model
best_model = grid_search.best_estimator_
y_pred_scaled = best_model.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

mae = mean_absolute_error(y_test, y_pred)
print(f"MAE of best model: {mae:.3f}")

Fitting 3 folds for each of 336 candidates, totalling 1008 fits
[CV] END activation=relu, alpha=1e-06, batch_size=32, early_stopping=True, hidden_layer_sizes=(100,), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.6s
[CV] END activation=relu, alpha=1e-06, batch_size=32, early_stopping=True, hidden_layer_sizes=(100,), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.7s
[CV] END activation=relu, alpha=1e-06, batch_size=32, early_stopping=True, hidden_layer_sizes=(100,), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   1.0s
[CV] END activation=relu, alpha=1e-06, batch_size=32, early_stopping=True, hidden_layer_sizes=(100,), learning_rate_init=0.0005, max_iter=500, solver=adam; total time=   0.9s
[CV] END activation=relu, alpha=1e-06, batch_size=32, early_stopping=True, hidden_layer_sizes=(100,), learning_rate_init=0.0005, max_iter=500, solver=adam; total time=   1.0s
[CV] END activation=relu, alpha=1e-06, batch_size=32, early_stop

### Training the ANN with PyTorch and optimal hyperparameters

Using PyTorch for compatibility with OMLT, but using the hyperparametes found previously to achieve a low MAE. 

Further exploration can lead to improvement of the model, which could alter the results obtained here. 

In [15]:
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import tempfile
import onnx
from omlt.io import write_onnx_model_with_bounds

# Raw data
train_features_raw = train_features.float()
test_features_raw = test_features.float()
train_y_raw = train_y.float().unsqueeze(1)
test_y_raw = test_y.float().unsqueeze(1)

# Neural Network Definition 
class FeedforwardNN(nn.Module):
    def __init__(self, input_dim=2, hidden_dims=(200, 100, 50, 25), output_dim=1):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dims[0]),
            nn.ReLU(),
            nn.Linear(hidden_dims[0], hidden_dims[1]),
            nn.ReLU(),
            nn.Linear(hidden_dims[1], hidden_dims[2]),
            nn.ReLU(),
            nn.Linear(hidden_dims[2], hidden_dims[3]),
            nn.ReLU(),
            nn.Linear(hidden_dims[3], output_dim)
        )
    def forward(self, x):
        return self.model(x)

model = FeedforwardNN()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# DataLoader
train_dataset = TensorDataset(train_features_raw, train_y_raw)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Loss & Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

# Training 
start = time.time()
for epoch in range(400):
    model.train()
    running_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
end = time.time()
time_nn = end - start

# Evaluation
model.eval()
with torch.no_grad():
    preds = model(test_features_raw.to(device)).cpu().numpy().flatten()
    targets = test_y_raw.numpy().flatten()
    mae_nn = np.mean(np.abs(preds - targets))


# Export to ONNX
dummy_input = torch.randn(1, 2)
input_bounds = [(1, 250), (1.275886, 65.523)]  # raw units: bar, kJ/mol

with tempfile.NamedTemporaryFile(suffix=".onnx", delete=False) as f:
    torch.onnx.export(
        model, dummy_input, f.name,
        input_names=["input"], output_names=["output"],
        dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}}
    )
    onnx_model = onnx.load(f.name)
    write_onnx_model_with_bounds(f.name, onnx_model, input_bounds)
    onnx_path = f.name


print(f'Model               Error (MAE)    Training time (s)')
print(f'Neural Network:       {mae_nn:.3f}          {time_nn:.2f}')

Model               Error (MAE)    Training time (s)
Neural Network:       1.916          56.35


### Solving the optimization problem with the embeded ANN surrogate

In [16]:
from omlt.io import load_onnx_neural_network_with_bounds
from omlt.neuralnet import FullSpaceNNFormulation

m = pyo.ConcreteModel()

# Define parameters
U = 6341.4
Cp = 1.507
Ks = 20
Ka = 150

# Define variables
m.TsIn = pyo.Var()
m.TsOut = pyo.Var()
m.TpIn = pyo.Var()
m.TpOut = pyo.Var()
m.Fp = pyo.Var(within=pyo.NonNegativeReals)
m.Fs = pyo.Var(within=pyo.NonNegativeReals)
m.A = pyo.Var(within=pyo.NonNegativeReals)
m.PsIn = pyo.Var(within=pyo.NonNegativeReals, bounds=(1, 250), initialize=100)
m.PsOut = pyo.Var(within=pyo.NonNegativeReals, bounds=(1, 250), initialize=100)
m.Q = pyo.Var(bounds=(0, 1e6), initialize=1e5)
m.HsIn = pyo.Var(within=pyo.NonNegativeReals, bounds=(1.275886, 65.523))
m.HsOut = pyo.Var(within=pyo.NonNegativeReals, bounds=(1.275886, 65.523))
m.LMTD = pyo.Var(bounds=(1e-3, 1000), initialize=10)
m.dT1 = pyo.Var(bounds=(0.01, None))
m.dT2 = pyo.Var(bounds=(0.01, None))

m.TsIn = pyo.Var(bounds=(300, 1000), initialize=500)
m.TsOut = pyo.Var(bounds=(300, 1000), initialize=500)

m.TpIn = pyo.Var(bounds=(300, 1000), initialize=500)
m.TpOut = pyo.Var(bounds=(300, 1000), initialize=500)

m.Fp = pyo.Var(bounds=(0, 1e5), initialize=1000)
m.Fs = pyo.Var(bounds=(0, 1e5), initialize=5000)

m.A = pyo.Var(bounds=(0.01, 1e4), initialize=100)  # Heat exchanger area

m.PsIn = pyo.Var(bounds=(1, 250), initialize=100)
m.PsOut = pyo.Var(bounds=(1, 250), initialize=100)


# Constraints for the model - physical laws and design equations for the HEX
m.heat_transfer = pyo.Constraint(expr=m.Q == U * m.A * m.LMTD)
m.LMTD_chenApp = pyo.Constraint(expr=m.LMTD == (m.dT1 * m.dT2 * (m.dT1 + m.dT2) / 2) ** (1 / 3))
m.dT1_eq = pyo.Constraint(expr=m.dT1 == m.TsOut - m.TpIn)
m.dT2_eq = pyo.Constraint(expr=m.dT2 == m.TsIn - m.TpOut)
m.proc_fluid_thermo = pyo.Constraint(expr=m.Q == m.Fp * Cp * (m.TpOut - m.TpIn))
m.steam_thermo = pyo.Constraint(expr=m.Q == -m.Fs * (m.HsOut - m.HsIn) * 1000 / 18)

# NN surrogate block - this is where we embed the surrogate
nn_definition = load_onnx_neural_network_with_bounds(onnx_path)
m.nn_outlet = OmltBlock()
formulation = FullSpaceNNFormulation(nn_definition)
m.nn_outlet.build_formulation(formulation)

# Connect surrogate inputs and outputs
m.connect_input_outlet_P = pyo.Constraint(expr=m.PsOut == m.nn_outlet.inputs[0])
m.connect_input_outlet_H = pyo.Constraint(expr=m.HsOut == m.nn_outlet.inputs[1])
m.connect_output_outlet = pyo.Constraint(expr=m.TsOut == m.nn_outlet.outputs[0])

# Bounds on TsOut - it's important to add the bounds not to get extrapolated values from the NN
m.TsOut.setlb(288.0)
m.TsOut.setub(930.0)

# Objective function - related to minimizing cost od HEX design
m.obj = pyo.Objective(expr=Ks * m.Fs + Ka * m.A)

# Fix variables
m.PsIn.fix(86.047)
m.TsIn.fix(866)
m.TpIn.fix(513.15)
m.TpOut.fix(831)
m.Fp.fix(310 * 3600)
m.HsIn.fix(65.233)

m.PsOut.set_value(100.0)
m.HsOut.set_value(30.0)
m.A.set_value(100.0)
m.Fs.set_value(5000.0)
m.TsOut.set_value(500.0)


# Solve
solver = pyo.SolverFactory("gams:baron")

start_time_m = time.time()
results = solver.solve(m, tee=True)
end_time_m = time.time()

time_m = end_time_m - start_time_m

WARNING (W1002): Setting Var 'nn_outlet.scaled_inputs[0]' to a numeric value
`0` outside the bounds (1.0, 250.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'nn_outlet.scaled_inputs[0]' to a numeric value
`0` outside the bounds (1.0, 250.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'nn_outlet.scaled_inputs[1]' to a numeric value
`0` outside the bounds (1.275886, 65.523).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
--- Job model.gms Start 04/04/25 19:48:56 41.5.0 2a5a4ddc DEX-DEG x86 64bit/macOS
--- Applying:
    /Library/Frameworks/GAMS.framework/Versions/41/Resources/gmsprmun.txt
--- GAMS Parameters defined
    Input /private/var/folders/ly/14n5x27j4x58m15svpw3z2zm0000gn/T/tmp6ue53n1j/model.gms
    Output /private/var/folders/ly/14n5x27j4x58m15svpw3z2zm0000gn/T/tmp6ue53n1j/output.lst
    ScrDir /private/var/folders/ly/14n5x27j4x58m15svpw3z2zm0000g

## Results comparison considering the neural network and tree surrogates

In [17]:
m_obj = m.obj.expr()
m_Fs = m.Fs.value / 3600  # Convert to kg/s
m_A = m.A.value

In [19]:
comparison_df = pd.DataFrame({
    "Metric": [
        "Objective Value ($)", 
        "Steam Flow (kg/s)", 
        "Heat Exchanger Area (m2)", 
        "Run time (s)", 
        "# of leaves", 
        "MAE (K)",
        "Training time (s)",
        "BARON Memory usage (MB)"
    ],
    "Original formulation": [
        f"{original_obj:.2e}", 
        f"{original_Fs:.2f}", 
        f"{original_A:.0f}", 
        "-", 
        "-", 
        "-", 
        "-",
        "-",
    ],
    "For linear tree surrogate": [
        f"{m1_obj:.2e}", 
        f"{m1_Fs:.2f}", 
        f"{m1_A:.0f}", 
        f"{time_m1:.2f}", 
        f"{linear_leaves}", 
        f"{linear_error:.2f}",
        f"{linear_time:.2f}",
        6 
    ],
    "For hyperplane tree surrogate (W = 3)": [
        f"{m3_obj:.2e}", 
        f"{m3_Fs:.2f}", 
        f"{m3_A:.0f}", 
        f"{time_m3:.2f}", 
        f"{hyperplane_leaves_3}", 
        f"{hyperplane_error_3:.2f}",
        f"{hyperplane_time_3:.2f}",
        8 
    ],
        "For hyperplane tree surrogate (W = 2)": [
        f"{m2_obj:.2e}", 
        f"{m2_Fs:.2f}", 
        f"{m2_A:.0f}", 
        f"{time_m2:.2f}", 
        f"{hyperplane_leaves_2}", 
        f"{hyperplane_error_2:.2f}",
        f"{hyperplane_time_2:.2f}",
        11 
    ],
        "For the neural network surrogate": [
        f"{m_obj:.2e}", 
        f"{m_Fs:.2f}", 
        f"{m_A:.0f}", 
        f"{time_m:.2f}", 
        f"{'-'}", 
        f"{mae_nn:.2f}",
        f"{time_nn:.2f}",
        7
    ]
})

comparison_df


,Metric,Original formulation,For linear tree surrogate,For hyperplane tree surrogate (W = 3),For hyperplane tree surrogate (W = 2),For the neural network surrogate
0,Objective Value ($),4.78e+06,4.76e+06,4.76e+06,4.76e+06,4.75e+06
1,Steam Flow (kg/s),60.31,60.08,60.09,60.09,60.02
2,Heat Exchanger Area (m2),2924,2915,2919,2921,2888
3,Run time (s),-,1.33,6.12,2.33,5096.89
4,# of leaves,-,337,168,174,-
5,MAE (K),-,0.30,0.30,0.28,1.92
6,Training time (s),-,2.99,6.94,2.61,56.35
7,BARON Memory usage (MB),-,6,8,11,7


Further discussion about the results in this table can be seen in the paper.